# Import necessary packages

In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [7]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [8]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [9]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
566,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,0,1,0,0,0,1
567,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
568,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,...,0,5,0,0,0,0,0,0,1,0
569,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,1,0,3,1,2,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10754,HOUSEHOLD_2_512_CA_4_validation,HOUSEHOLD_2_512,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,2,1,0,0,0,0
10755,HOUSEHOLD_2_513_CA_4_validation,HOUSEHOLD_2_513,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
10756,HOUSEHOLD_2_514_CA_4_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
10757,HOUSEHOLD_2_515_CA_4_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [10]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Household'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [11]:
len(sales_intermittent)

1493

In [12]:
len(sales_lumpy)

178

In [13]:
len(sales_erratic)

126

In [14]:
len(sales_smooth)

2391

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [15]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
593,HOUSEHOLD_1_029_CA_1_validation,HOUSEHOLD_1_029,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,4,2,0,2,0,0,1,0
627,HOUSEHOLD_1_065_CA_1_validation,HOUSEHOLD_1_065,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,0,3,2,0,0,0,1,1
673,HOUSEHOLD_1_113_CA_1_validation,HOUSEHOLD_1_113,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,1,2,3,7,1,1,2,2,2,2
692,HOUSEHOLD_1_132_CA_1_validation,HOUSEHOLD_1_132,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,0,0,2,0,3,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10208,HOUSEHOLD_1_506_CA_4_validation,HOUSEHOLD_1_506,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,1,0,0,0,...,3,0,0,2,1,1,0,1,0,2
10481,HOUSEHOLD_2_239_CA_4_validation,HOUSEHOLD_2_239,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,1,1,1,...,2,1,0,3,0,0,1,2,2,1
10543,HOUSEHOLD_2_301_CA_4_validation,HOUSEHOLD_2_301,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,1,2,1,1,...,0,0,0,3,0,0,0,0,0,3
10689,HOUSEHOLD_2_447_CA_4_validation,HOUSEHOLD_2_447,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,11,...,0,6,0,0,0,0,4,0,1,1


# User-defined functions to calculate Metrics and Croston algorithm

In [16]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [17]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [18]:
sales_pattern = sales_erratic.copy()

In [19]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [20]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [21]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [25]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 100

In [26]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [27]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [28]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [29]:
len(sample_list_filtered)

77

# Hyperparameter tuning for Croston

In [30]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [31]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOUSEHOLD_1_374_CA_2
Progressing: 0.0 %
Currently Running: HOUSEHOLD_1_065_CA_2
Progressing: 1.3 %
Currently Running: HOUSEHOLD_1_321_CA_4
Progressing: 2.6 %
Currently Running: HOUSEHOLD_1_172_CA_1
Progressing: 3.9 %
Currently Running: HOUSEHOLD_1_133_CA_1
Progressing: 5.19 %
Currently Running: HOUSEHOLD_1_008_CA_2
Progressing: 6.49 %
Currently Running: HOUSEHOLD_1_230_CA_1
Progressing: 7.79 %
Currently Running: HOUSEHOLD_1_113_CA_1
Progressing: 9.09 %
Currently Running: HOUSEHOLD_1_229_CA_3
Progressing: 10.39 %
Currently Running: HOUSEHOLD_1_357_CA_3
Progressing: 11.69 %
Currently Running: HOUSEHOLD_1_493_CA_2
Progressing: 12.99 %
Currently Running: HOUSEHOLD_2_053_CA_2
Progressing: 14.29 %
Currently Running: HOUSEHOLD_1_341_CA_1
Progressing: 15.58 %
Currently Running: HOUSEHOLD_1_277_CA_2
Progressing: 16.88 %
Currently Running: HOUSEHOLD_2_447_CA_4
Progressing: 18.18 %
Currently Running: HOUSEHOLD_1_379_CA_4
Progressing: 19.48 %
Currently Running: HOUSEHOLD_2_440_C

In [32]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOUSEHOLD_1_374_CA_2,9.0,5.653801,0.05
1,HOUSEHOLD_1_374_CA_2,4.0,5.807638,0.05
2,HOUSEHOLD_1_374_CA_2,8.0,5.724198,0.05
3,HOUSEHOLD_1_374_CA_2,6.0,5.829653,0.05
4,HOUSEHOLD_1_374_CA_2,6.0,5.837576,0.05
...,...,...,...,...
11083,HOUSEHOLD_1_504_CA_3,1.0,10.215931,0.90
11084,HOUSEHOLD_1_504_CA_3,7.0,1.995576,0.90
11085,HOUSEHOLD_1_504_CA_3,4.0,6.495508,0.90
11086,HOUSEHOLD_1_504_CA_3,8.0,4.249753,0.90


# Summay all metrics

In [33]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [34]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0415610150999972 minutes


In [35]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_008_CA_2,0.05,1.148114,34.3653,40.2520,38.8449,1.804179,1.872856
1,HOUSEHOLD_1_008_CA_2,0.10,0.669249,20.0319,23.0286,30.7931,1.051677,1.377476
2,HOUSEHOLD_1_008_CA_2,0.15,0.623489,18.6623,21.4260,31.3024,0.979769,1.399059
3,HOUSEHOLD_1_008_CA_2,0.20,0.623499,18.6626,21.3730,31.8538,0.979784,1.436788
4,HOUSEHOLD_1_008_CA_2,0.25,0.638328,19.1064,21.8127,32.4131,1.003087,1.468853
...,...,...,...,...,...,...,...,...
1381,HOUSEHOLD_2_512_CA_4,0.70,0.908529,88.9988,82.6402,110.8404,3.114958,3.633662
1382,HOUSEHOLD_2_512_CA_4,0.75,0.936162,91.7057,84.2120,114.4359,3.209699,3.711860
1383,HOUSEHOLD_2_512_CA_4,0.80,0.963067,94.3413,85.7065,117.9348,3.301944,3.792678
1384,HOUSEHOLD_2_512_CA_4,0.85,0.988912,96.8730,87.0914,121.2749,3.390554,3.875891


# Check what products has unexpected metrics result

In [36]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [37]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [38]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
270,HOUSEHOLD_1_172_CA_1,0.05,0.670050,99.6948,NaN,98.9279,6.604782,11.788837
271,HOUSEHOLD_1_172_CA_1,0.10,0.669295,99.5825,NaN,98.7044,6.597339,11.779615
272,HOUSEHOLD_1_172_CA_1,0.15,0.668319,99.4372,NaN,98.4318,6.587712,11.768342
273,HOUSEHOLD_1_172_CA_1,0.20,0.667045,99.2476,NaN,98.0950,6.575155,11.754418
274,HOUSEHOLD_1_172_CA_1,0.25,0.665368,98.9982,NaN,97.6733,6.558631,11.737045
275,HOUSEHOLD_1_172_CA_1,0.30,0.663141,98.6667,NaN,97.1375,6.536672,11.715156
276,HOUSEHOLD_1_172_CA_1,0.35,0.660149,98.2217,NaN,96.4464,6.507184,11.687336
277,HOUSEHOLD_1_172_CA_1,0.40,0.656089,97.6175,NaN,95.5413,6.467158,11.651747
278,HOUSEHOLD_1_172_CA_1,0.45,0.650517,96.7886,NaN,94.3375,6.412243,11.606112
279,HOUSEHOLD_1_172_CA_1,0.50,0.642799,95.6401,NaN,92.7133,6.336159,11.547916


In [39]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [40]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [41]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [42]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [43]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 0.0 %
Percentage of unexpected values of WMAPE is: 0.0 %
Percentage of unexpected values of SMAPE is: 3.896103896103896 %
Percentage of unexpected values of MAPE is: 0.0 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [44]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [45]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,4.985639,509.658938,71.737686,133.338758,6.524559,7.359358
0.10,4.496485,460.438261,66.873808,124.766015,5.983435,6.762962
0.15,4.107703,419.708638,64.569312,119.734177,5.541599,6.372904
0.20,3.696601,374.755449,63.507223,115.172153,5.137174,6.031805
0.25,3.292734,330.100986,62.859412,110.720015,4.819052,5.734271
0.30,2.923570,289.018658,62.451080,106.586834,4.615528,5.530222
0.35,2.596789,252.677645,62.188128,102.812476,4.422536,5.398217
0.40,2.318189,221.360297,62.250865,99.658750,4.302174,5.301891
0.45,2.083355,194.569605,62.540177,97.020934,4.249656,5.232097


In [46]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.9
The optimum Alpha based on Mean WMAPE is: 0.9
The optimum Alpha based on Mean SMAPE is: 0.35
The optimum Alpha based on Mean MAPE is: 0.9
The optimum Alpha based on Mean MAE is: 0.5
The optimum Alpha based on Mean RMSE is: 0.55


### Get MEDIAN metrics value of each Alpha

In [47]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,0.844357,55.08090,65.32965,56.96550,3.191328,3.855355
0.10,0.799341,52.41840,62.18315,52.46440,3.015453,3.847740
0.15,0.782729,52.95210,60.86005,51.90750,2.977048,3.722713
0.20,0.779462,52.59940,58.90250,52.03730,2.892609,3.566945
0.25,0.776868,53.00460,58.91375,53.96655,2.874733,3.431497
0.30,0.778778,52.85860,58.90585,55.75460,2.877794,3.454350
0.35,0.778432,52.06260,58.42170,56.05080,2.877140,3.495544
0.40,0.792038,53.19505,57.54050,57.02250,2.909575,3.542868
0.45,0.821284,53.64495,58.67640,56.59770,2.925822,3.594939


In [48]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.25
The optimum Alpha based on Median of WMAPE is: 0.35
The optimum Alpha based on Median of SMAPE is: 0.4
The optimum Alpha based on Median of MAPE is: 0.15
The optimum Alpha based on Median of MAE is: 0.25
The optimum Alpha based on Median of RMSE is: 0.25


# Run Croston model after decide best paramaters

In [50]:
best_alpha = 0.9

In [51]:
dataframe_weekly

,wm_yr_wk,HOUSEHOLD_1_001_CA_1,HOUSEHOLD_1_002_CA_2,HOUSEHOLD_1_004_CA_2,HOUSEHOLD_1_008_CA_2,HOUSEHOLD_1_028_CA_2,HOUSEHOLD_1_029_CA_1,HOUSEHOLD_1_033_CA_3,HOUSEHOLD_1_043_CA_3,HOUSEHOLD_1_065_CA_1,...,HOUSEHOLD_2_447_CA_4,HOUSEHOLD_2_448_CA_3,HOUSEHOLD_2_452_CA_3,HOUSEHOLD_2_460_CA_3,HOUSEHOLD_2_498_CA_1,HOUSEHOLD_2_498_CA_3,HOUSEHOLD_2_499_CA_2,HOUSEHOLD_2_511_CA_3,HOUSEHOLD_2_512_CA_4,HOUSEHOLD_2_513_CA_1
0,11101,4.0,7.0,2.0,0.0,7.0,4.0,16.0,0.0,16.0,...,1.0,0.0,6.0,1.0,0.0,0.0,0.0,31.0,8.0,5.0
1,11102,0.0,5.0,0.0,5.0,6.0,9.0,15.0,0.0,26.0,...,0.0,0.0,8.0,4.0,0.0,0.0,0.0,16.0,1.0,0.0
2,11103,2.0,2.0,0.0,5.0,0.0,5.0,7.0,0.0,16.0,...,3.0,0.0,1.0,1.0,0.0,0.0,0.0,25.0,0.0,7.0
3,11104,0.0,1.0,0.0,4.0,1.0,6.0,12.0,0.0,14.0,...,3.0,0.0,9.0,2.0,0.0,0.0,0.0,17.0,0.0,0.0
4,11105,2.0,7.0,0.0,11.0,5.0,1.0,7.0,0.0,20.0,...,4.0,0.0,3.0,1.0,0.0,0.0,0.0,14.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,1.0,1.0,9.0,6.0,5.0,12.0,2.0,0.0,21.0,...,5.0,0.0,1.0,1.0,0.0,0.0,0.0,19.0,6.0,0.0
269,11609,0.0,1.0,5.0,6.0,13.0,12.0,2.0,0.0,28.0,...,2.0,0.0,4.0,2.0,0.0,0.0,0.0,9.0,0.0,4.0
270,11610,1.0,9.0,11.0,5.0,3.0,9.0,3.0,0.0,28.0,...,1.0,0.0,6.0,1.0,0.0,0.0,0.0,20.0,1.0,2.0
271,11611,2.0,4.0,0.0,3.0,7.0,5.0,13.0,0.0,24.0,...,5.0,0.0,6.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0


In [52]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [53]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [54]:
df_result_best_params

,Product,Actual Data,Forecast
0,HOUSEHOLD_1_001_CA_1,2.0,0.504842
1,HOUSEHOLD_1_001_CA_1,1.0,1.728264
2,HOUSEHOLD_1_029_CA_1,5.0,9.304508
3,HOUSEHOLD_1_029_CA_1,9.0,5.430451
4,HOUSEHOLD_1_065_CA_1,24.0,27.937173
...,...,...,...
247,HOUSEHOLD_2_301_CA_4,1.0,4.470942
248,HOUSEHOLD_2_447_CA_4,5.0,1.126736
249,HOUSEHOLD_2_447_CA_4,2.0,4.612642
250,HOUSEHOLD_2_512_CA_4,1.0,0.773286


In [60]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [61]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_001_CA_1,1.111711,74.1141,86.3840,73.7922,1.111711,1.175982
1,HOUSEHOLD_1_002_CA_2,inf,57.7909,39.4337,57.7909,2.311635,2.986773
2,HOUSEHOLD_1_004_CA_2,inf,inf,200.0000,NaN,10.440239,10.440239
3,HOUSEHOLD_1_008_CA_2,inf,38.4828,29.2976,38.4828,1.154485,1.491665
4,HOUSEHOLD_1_028_CA_2,1.192469,39.7490,42.6506,38.9055,2.384938,2.483499
...,...,...,...,...,...,...,...
121,HOUSEHOLD_2_498_CA_3,NaN,NaN,NaN,NaN,0.000000,0.000000
122,HOUSEHOLD_2_499_CA_2,NaN,NaN,NaN,NaN,0.000000,0.000000
123,HOUSEHOLD_2_511_CA_3,1.849561,142.2739,89.3521,206.8862,9.247806,10.888232
124,HOUSEHOLD_2_512_CA_4,0.533279,71.1039,80.5566,51.7309,2.133116,2.860866


In [62]:
df_result_final.isnull().sum()

Product     0
MASE       22
WMAPE      22
SMAPE      22
MAPE       28
MAE         0
RMSE        0
dtype: int64

In [63]:
df_result_final.MAPE.mean()

90.62987959183675

In [64]:
df_result_final.to_csv('Croston_Erratic_Metrics.csv')